# Lista 8 de Cálculo Numérico

In [8]:
using LinearAlgebra # Para uso da função norm

## Questão 1.

### a. Compressão (Thierry)

### Compressão de Dados em Matrizes

A ideia da compressão de dados é encontrarmos vetores base dentro da matriz tal que forneçam uma base para a composição de todos os dados. Porém, na vida real precisamos encontrar aproximações, visto que o uso de números reais acabam dificultando uma base L.I de poucos vetores representarem muitos dados.

Na ideia simples, transformamos a matriz em uma multiplicação de duas matrizes, tal que a primeira seja os vetores base e a segunda os coeficientes das combinações lineares.

Com a preocupação de se adequar aos números reais, começamos a analisar isso pelo erro da matriz original com a nova multiplicação das duas matrizes. Para quantificar o erro, usamos a ideia de norma de matriz, que é dada pela soma das normas de cada vetor (colunas) da matriz.

Então, temos que dado a matriz A e o vetor b, precisamos determinar c tal que 

$$min ||A - [b][c^T]|| \longrightarrow min ||[(a_1)(a_2)...(a_n)] - [c_1(b) c_2(b) ... c_n(b)]|| \longrightarrow ||(a_1 - c_1b)(a_2-c_2b)...(a_n-c_nb)||$$

Visto que $e_n = a_n - c_nb \space \bot \space c_nb$, pelo produto interno temos que $e_n^Tc_nb = 0 \longrightarrow (a_n - c_nb)^Tc_nb = 0$. Aplicando a transposta tal que $(M.Q)^T = Q^T.M^T$ e $(M - Q)^T = M^T - Q^T$ obtemos:

$$(a_n - c_nb)^Tc_nb = 0 \longrightarrow (a_n^T - (c_nb)^T)c_nb = 0 \longrightarrow a_n^Tc_nb - c_n^2b^Tb = 0 \longrightarrow a_n^Tc_nb = c_n^2b^Tb $$

Desconsiderando o caso em que $c_n = 0$ pois é um caso muito trivial, podemos dividir a equação em ambos os lados por $c_n$, resultando em:

$$a_n^Tb = c_nb^Tb \longrightarrow c_n = \frac{a_n^Tb}{b^Tb}$$

A partir da análise feita acima podemos escrever um código em Julia para achar o vetor c:

In [2]:
function melhores_coords(A,b) #retorna c tal que norm(A-b*c') é mínima
    m, n = size(A)
    A = 1.0 * A # multiplica-se por 1.0 para evitar erros de operações matriciais entre inteiros e float's.
    c = zeros(n)    
    c = A'*(b/(b'*b)) # Operação para achar o vetor c que minimiza a distância perpendicular à reta.
    return c
end

melhores_coords (generic function with 1 method)

In [3]:
A=[1.0001 10.001 3;2.01 20.3 6;3.003 30 9]

3×3 Matrix{Float64}:
 1.0001  10.001  3.0
 2.01    20.3    6.0
 3.003   30.0    9.0

In [4]:
b=[1;2;3]

3-element Vector{Int64}:
 1
 2
 3

In [5]:
c=melhores_coords(A,b)

3-element Vector{Float64}:
  1.0020785714285714
 10.04292857142857
  3.0

In [9]:
norm(A-b*c')

0.2534747564213406

In [10]:
function acha_melhor_minimizacao(A)
    n,m = size(A)

    bv = randn(n)
    cv = randn(m)
    
    bw = zeros(m)
    cw = zeros(n)
    
    for i in 1:50
        cv=melhores_coords(A, bv)
        bw=cv
        cw=melhores_coords(A', bw)
        bv=cw
    end
    
    return bv,cv
end

acha_melhor_minimizacao (generic function with 1 method)

In [11]:
function compressao(A,nível)
    m,n=size(A)
    B=zeros(m,nível)
    C=zeros(n,nível)
    
    for i=1:nível
        B[:,i], C[:,i] = acha_melhor_minimizacao(A)
        A= A - B[:,i]*C[:,i]'
    end
    return B,C
end

compressao (generic function with 1 method)

In [12]:
A = randn(3,4)

3×4 Matrix{Float64}:
 -0.160816   2.31884    -0.435174  0.392454
 -1.75984   -1.01735    -0.3824    0.224054
  0.872807  -0.0267881  -0.105852  2.90635

In [13]:
B,C = compressao(A,3)

([-0.5662597655101601 -0.797658084119734 2.7735692711193933; 0.42058161168373775 0.5268095302073903 4.114842035801667; -1.554043288436568 0.4332235000686877 0.10299984005737532], [-0.6885533763638839 -0.3819492758450379 -0.3084043681711147; -0.5834376457095105 -2.17636199699255 0.091025958433934; 0.08586521400937591 0.09061557608269888 -0.11330933272319452; -1.5946615140750595 0.9660637667964911 0.09376006549304033])

In [14]:
norm(A-B*C')

5.110461207377096e-16

In [15]:
# Chance pequena de existirem colunas Linearmente Dependentes
N = 20
A = randn(N, N)
for i in 1:N
    B,C = compressao(A, i)
    print(i, ": ", norm(A-B*C'), "\n")
end

1: 17.74264414821714
2: 15.981264749704538
3: 14.45888837473977
4: 12.926533969059475
5: 11.407449193163298
6: 10.157298165931056
7: 9.041313255542105
8: 7.884793950698458
9: 6.922988960680465
10: 5.870362946719035
11: 5.047332584439586
12: 4.144838076839678
13: 3.246690352481172
14: 2.381988228048469
15: 1.5442076968775071
16: 1.0953321105055063
17: 0.7508102837520597
18: 0.33170693393622214
19: 0.08747632377829191
20: 4.950096130911357e-15


In [16]:
N = 20
A = zeros(N, N)
for i in 1:N
    A[:, i] .= i
end
for i in 1:10
    B,C = compressao(A, i)
    print(norm(A-B*C'), "\n")
end

3.445628625312982e-14
0.0
0.0
NaN
NaN
NaN
NaN
0.0
NaN
NaN


### b. Gauss-Jacobi (Vinicius)

### c. Gauss-Seidel (Vinicius)

## Questão 2. (Thierry)

## Questão 3. (Chris)

## Questão 4. (Thierry)

## Questão 5. (Chris)

## Questão 6. (Luan)

## Questão 7. (Vinicius)

## Questão 8. (Luan)